In [1]:
import numpy as np # Library untuk komputasi numerik, termasuk array multidimensional dan berbagai fungsi matematika
import pandas as pd # Library yang berguna dalam analisis data dan manipulasi data tabular
from sklearn.tree import DecisionTreeClassifier # import DT, implementasi dari algoritma pohon keputusan (decision tree) yang digunakan untuk tugas klasifikasi
from sklearn.ensemble import RandomForestClassifier # import RandomForest, implementasi dari algoritma "Random Forest" yang digunakan untuk tugas klasifikasi
from sklearn.model_selection import train_test_split # Function yang berguna dalam pembagian dataset menjadi subset pelatihan dan pengujian
from sklearn.metrics import accuracy_score, classification_report # accuracy_score berguna untuk mengukur akurasi model klasifikasi dan classification_report merupakan fungsi yang menghasilkan laporan klasifikasi yang lebih rinci, termasuk presisi, recall, dan F1-score untuk setiap kelas dalam masalah klasifikasi
from sklearn.preprocessing import LabelEncoder # Kebutuhan encoding label
from sklearn.preprocessing import StandardScaler #  digunakan untuk mengubah distribusi dari setiap atribut (fitur) dalam dataset sehingga memiliki rata-rata nol dan deviasi standar satu

**Tugas 1**
Terdapat dataset mushroom. Berdasarkan dataset yang tersebut, bandingkan peforma antara algoritma Decision Tree dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [2]:
# Load data
df = pd.read_csv('data/mushrooms.csv') # Membaca data dari file CSV dan menyimpannya dalam sebuah DataFrame

df.head() # Menampilkan 5 baris data teratas

class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w         p          w   
1                      w                      w         p          w   
2                      w                      w         p          w   
3                      w                      w         p          w   
4                      w                      w         p          w   

  ring-number ring-type spore-print-color population habitat  
0           o         p                 k          s       u  
1           o         p                 n          n       g  
2           o         p                 n          n       m  
3           o         p                 k          s       u  
4           o         e                 n          a       g  

[5 rows x 23 columns]

In [3]:
# Cek kolom null
df.isnull().sum() # menghitung jumlah nilai-nilai yang hilang (NaN) dalam setiap kolom DataFrame df

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [5]:
X = df.iloc[:, :-1]
y = df['habitat']

# encode label
ec = LabelEncoder() # membuat objek LabelEncoder
# X = ec.fit_transform(X)
for column in X.columns:
    if X[column].dtype == 'object':
        X[column] = ec.fit_transform(X[column])
        
y = ec.fit_transform(y)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_24480\827958006.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = ec.fit_transform(X[column])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_24480\827958006.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = ec.fit_transform(X[column])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_24480\827958006.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

sc = StandardScaler() # Membuat objek StandardScaler

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train) # Menstandarisasi X_train
X_test_std = sc.transform(X_test) # Menstandarisasi y_train

Decision Tree Classifier

In [7]:
dt = DecisionTreeClassifier() # membuat model Decision Tree Classifier

from sklearn.model_selection import GridSearchCV
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Membuat objek GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='accuracy')

# Melakukan pencarian parameter terbaik
grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik
best_params = grid_search.best_params_
print("Parameter terbaik:", best_params)

Parameter terbaik: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'splitter': 'random'}


In [8]:
best_dt = DecisionTreeClassifier(**best_params)
best_dt.fit(X_train, y_train)
y_pred_dt = best_dt.predict(X_test)
accuracy = best_dt.score(X_test, y_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.56
Test set accuracy: 0.56


Random Forest Classifier

In [9]:
# Membuat model Random Forest
rf = RandomForestClassifier()

# Definisi hyperparameter yang akan diuji
param_grid = {
    'n_estimators': [50, 100, 150],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Membuat objek GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')

# Melakukan pencarian parameter terbaik
grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik
best_params = grid_search.best_params_
print("Parameter terbaik:", best_params)

Parameter terbaik: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}


In [10]:
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)
y_pred_rf = best_rf.predict(X_test)

#  menghitung set accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Test set accuracy: {:.2f}".format(acc_rf))
print(f"Test set accuracy: {acc_rf}")

Test set accuracy: 0.51
Test set accuracy: 0.5095384615384615


**Tugas 2**
Terdapat dataset mushroom. Berdasarkan dataset tersebut, bandingkan peforma antara algoritma Decision Tree dan AdaBoost. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

Decision Tree Classifier

In [11]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Membuat objek GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='accuracy')

# Melakukan pencarian parameter terbaik
grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik
best_params = grid_search.best_params_
print("Parameter terbaik:", best_params)

Parameter terbaik: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'random'}


In [12]:
best_dt = DecisionTreeClassifier(**best_params)
best_dt.fit(X_train, y_train)
y_pred_dt = best_dt.predict(X_test)
accuracy = best_dt.score(X_test, y_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.56
Test set accuracy: 0.563076923076923


AdaBoost Classifier

In [13]:
from sklearn.ensemble import AdaBoostClassifier # import AdaBoost, algoritma ensemble learning yang dapat digunakan untuk meningkatkan kinerja model klasifikasi

ab = AdaBoostClassifier(base_estimator=None)

param_grid = {
    'n_estimators': [50, 100, 200],  # Jumlah estimator (pohon) dalam ensemble
    'learning_rate': [0.01, 0.1, 1.0],  # Tingkat pembelajaran
    'algorithm': ['SAMME', 'SAMME.R']  # Algoritma yang digunakan
}

grid_search = GridSearchCV(ab, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Melakukan pencarian parameter terbaik
grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik
best_params = grid_search.best_params_
print("Parameter terbaik:", best_params)

c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Parameter terbaik: {'algorithm': 'SAMME', 'learning_rate': 1.0, 'n_estimators': 200}


In [14]:
best_ab = AdaBoostClassifier(**best_params)
best_ab.fit(X_train, y_train)
y_pred_ab = best_ab.predict(X_test)
accuracy = best_ab.score(X_test, y_test)

#  menghitung set accuracy
acc_ab = accuracy_score(y_test, y_pred_ab)
print("Test set accuracy: {:.2f}".format(acc_ab))
print(f"Test set accuracy: {acc_ab}")

Test set accuracy: 0.62
Test set accuracy: 0.6233846153846154


**Tugas 3**
Dengan menggunakan dataset diabetes, buatlah ensemble voting dengan algoritma:
1. Logistic Regression
2. SVM kernel polynomial
3. Decission Tree

In [15]:
from sklearn.ensemble import VotingClassifier

# Load Data
dbt = pd.read_csv('data/diabetes.csv') # Membaca data dari file CSV dan menyimpannya dalam sebuah DataFrame

dbt.head() # Menampilkan 5 baris data teratas

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1

In [17]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer #  berguna untuk mengatasi missing values (nilai yang hilang) dalam data 

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False) # mengatasi nilai-nilai 0 (nol) dalam dataset dengan menggantinya dengan nilai rata-rata dari kolom
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'] # variabel untuk menyimpan nama fitur/kolom

dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns]) # menggantikan nilai-nilai yang hilang dalam dataset dengan nilai rata-rata dari kolom

In [18]:
X = dbt[feature_columns] # membuat variabel X yang akan berisi fitur-fitur (features) dari dataset dbt.feature_columns
y = dbt.Outcome # membuat variabel y yang akan berisi fitur-fitur (features) dari dataset dbt dengan kolom Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # membagi data menjadi dua set: data pelatihan dan data pengujian dengan proporsi 70% data pelatihan dan 30% data pengujian

In [19]:
from sklearn.preprocessing import StandardScaler #  digunakan untuk mengubah distribusi dari setiap atribut (fitur) dalam dataset sehingga memiliki rata-rata nol dan deviasi standar satu

sc = StandardScaler() # Membuat objek StandardScaler

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train) # Menstandarisasi X_train
X_test_std = sc.transform(X_test) # Menstandarisasi y_train

In [21]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Definisikan model-model yang akan digunakan
clf1 = LogisticRegression()
clf2 = SVC(kernel='poly', degree=3)
clf3 = DecisionTreeClassifier()

# Buat ensemble voting dengan hard voting (majority vote)
voting = VotingClassifier(estimators=[('Logistic Regression', clf1), ('SVM Polynomial', clf2), ('Decision Tree', clf3)], voting='hard')

# Latih model
voting.fit(X_train_std, y_train)

# Prediksi
y_pred_voting = voting.predict(X_test_std)

# Evaluasi akurasi pada data uji
accuracy = accuracy_score(y_test, y_pred_voting)

# Print hasil evaluasi
print('Ensemble Voting with Logistic Regression, SVM Polynomial, and Decision Tree')
print("Test set accuracy: {:.2f}".format(accuracy))

Ensemble Voting with Logistic Regression, SVM Polynomial, and Decision Tree
Test set accuracy: 0.74
